# Descrição

*Código base para classificação de imagens usando redes convolucionais
*Bibliotecas
*Carregar Bases de dados
*Construir Modelos
*Treinamento
*Avaliação
*Hold out
Validação Cruzada

# Bibliotecas

In [8]:
import os
import zipfile
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from albumentations.core.composition import OneOf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Carregar Bases dados

In [1]:
#Conexão com Colab
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
from google.colab import files

Mounted at /content/gdrive


Carregar Base GAMMA

In [7]:
#Descompactar arquivo e salvar imagens (X) e CSV (LABELS)
path_to_dataset = '/content/gdrive/MyDrive/Doutorado/Bases/GAMMA_training_data/multi-modality_images/dataset_fundus/dataset_fundus.zip'

import zipfile

def descompactar_arquivo(zip_path, destino):
    """Descompacta um arquivo .zip e salva em uma pasta específica."""
    if not os.path.exists(destino):
        os.makedirs(destino)  # Cria a pasta de destino se não existir

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print(f"Arquivos extraídos para: {destino}")

# Exemplo de uso
zip_path = path_to_dataset  # Substitua pelo caminho do seu arquivo .zip
destino = "/content/Dataset GAMMA 2"  # Substitua pelo caminho onde deseja salvar os arquivos

descompactar_arquivo(zip_path, destino)


Arquivos extraídos para: /content/Dataset GAMMA 2


In [19]:
def carregar_imagens_e_labels(dataset_path):
    """Carrega imagens da pasta e os labels do arquivo train.csv"""
    IMG_SIZE = (128, 128)
    # Caminho do CSV com os labels
    csv_path = os.path.join(dataset_path, "train.csv")

    # Lendo os labels do CSV
    df = pd.read_csv(csv_path)

    # Criando listas para armazenar imagens e labels
    X = []  # Lista de imagens (arrays)
    y = []  # Lista de rótulos (classes)

    # Percorrendo cada linha do CSV
    for _, row in df.iterrows():
        #img_name = row.iloc[0]  # Nome do arquivo da imagem (ex: "0001.jpg")
        img_name = str(row.iloc[0]).zfill(4) + ".jpg"
        label = row.iloc[1:].values  # Labels (non, early, mid_advanced)

        # Determinando a classe (índice do label 1)
        class_id = np.argmax(label)  # Ex: [1, 0, 0] → 0 | [0, 0, 1] → 2

        # Caminho completo da imagem
        img_path = os.path.join(dataset_path, img_name)
        #print(img_path)

        # Verifica se a imagem existe antes de carregar
        if os.path.exists(img_path):
            # Carrega a imagem em escala de cinza (ou use cv2.IMREAD_COLOR para RGB)
            #img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Redimensiona para um tamanho fixo (opcional, ex: 128x128)
            img = cv2.resize(img, IMG_SIZE)

            # Normaliza os pixels (0 a 1)
            img = img.astype(np.float32) / 255.0

            # Adiciona à lista
            X.append(img)
            y.append(class_id)
        else:
            print(f"⚠️ Aviso: Imagem {img_name} não encontrada.")

    # Convertendo listas para arrays NumPy
    #X = np.array(X).reshape(-1, 128, 128, 1)  # Mantém formato para CNNs
    X = np.array(X)
    y = np.array(y)

    print(f"✅ {len(X)} imagens carregadas com sucesso!")
    return X, y

# Caminho para a pasta do dataset
dataset_path = "/content/Dataset GAMMA 2"

# Chamando a função para carregar imagens e labels
X, y = carregar_imagens_e_labels(dataset_path)

# Verificando as dimensões
print(f"Formato de X: {X.shape}, Formato de y: {y.shape}")


✅ 100 imagens carregadas com sucesso!
Formato de X: (100, 128, 128, 3), Formato de y: (100,)


# Construir Modelos